In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
label2id = {'negative':0,'positive':1}

train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train = train.replace({"label": label2id})
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test = test.replace({"label": label2id})
test.fillna('', inplace=True)

/tmp/ipykernel_8361/2093326092.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train = train.replace({"label": label2id})
/tmp/ipykernel_8361/2093326092.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test = test.replace({"label": label2id})


In [4]:
train

,label,text,diff
0,0,Fix macro names - related to #448,diff --git a/src/ciphers/aes/aes.c b/src/ciphe...
1,0,9.0.4,diff --git a/package.json b/package.json\ninde...
2,1,https://github.com/ImageMagick/ImageMagick/iss...,diff --git a/coders/png.c b/coders/png.c\ninde...
3,0,test: incr. disk size for TEST 35 ISCSI-MULTI,diff --git a/test/TEST-35-ISCSI-MULTI/test.sh ...
4,0,git-svn-id: https://plugins.svn.wordpress.org/...,diff --git a/inc/define.php b/inc/define.php\n...
5,0,[JBPM-9474] Produce events to Kafka by Signal ...,diff --git a/jbpm-bpmn2/src/main/java/org/jbpm...
6,1,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
7,1,Normalize resource URL in ResourceServlet\n\nI...,diff --git a/spring-webmvc/src/main/java/org/s...
8,0,Merge branch '1.8.x',diff --git a/src/ClientCommand.cpp b/src/Clien...
9,0,Change distribution URL.\n\ngit-svn-id: https:...,diff --git a/c/xml-security-c.spec b/c/xml-sec...


In [5]:
len(train)

50

In [6]:
train['label'].value_counts()

label
0    31
1    19
Name: count, dtype: int64

In [7]:
test['label'].value_counts()

label
0    6316
1    3746
Name: count, dtype: int64

In [8]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
# model_id = "../../sentence-transformers/paraphrase-mpnet-base-v2"
model = SentenceTransformer('../../sentence-transformers/paraphrase-mpnet-base-v2')

In [9]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [10]:
test

,label,text,diff,embeddings
0,0,Merge pull request #371 from rhenium/ky/hmac-e...,diff --git a/ext/openssl/extconf.rb b/ext/open...,"[-0.0071238917, -0.02382523, 0.08038095, -0.03..."
1,1,Additional check when unpacking archives. Cont...,diff --git a/hadoop-common-project/hadoop-comm...,"[0.040803984, 0.38105646, -0.048270673, 0.0595..."
2,0,Allow lists of trackingid footers\n\nGerrit al...,diff --git a/templates/gerrit.config.erb b/tem...,"[-0.08971211, 0.21257286, -0.020339478, 0.1116..."
3,0,Add test for opening a NeXT sound file with ze...,diff --git a/ChangeLog b/ChangeLog\nindex aaa5...,"[-0.1575543, 0.05420915, 0.026407806, -0.03680..."
4,1,git-prompt.sh: don't put unsanitized branch na...,diff --git a/contrib/completion/git-prompt.sh ...,"[-0.100363135, -0.06585043, -0.06453755, 0.056..."
...,...,...,...,...
10057,1,Fix hostname verification using the deprecated...,diff --git a/rt/transports/http/src/main/java/...,"[-0.098657325, 0.12404662, 0.018913269, -0.124..."
10058,0,mod_mam: Add missing semicolon,diff --git a/src/mod_mam.erl b/src/mod_mam.erl...,"[-0.009726456, -0.11668825, -0.024219565, -0.0..."
10059,0,Create an initial project,diff --git a/.gitignore b/.gitignore\nnew file...,"[-0.21997099, 0.08384434, -0.060743522, -0.108..."
10060,0,Merge pull request #501 from timgates42/bugfix...,diff --git a/src/config.c b/src/config.c\ninde...,"[0.012738513, -0.0528628, -0.011895727, 0.0169..."


In [11]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [13]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))

Logistic Regression Precision: 0.7577239120843295
Logistic Regression Recall: 0.7617769827072153
Logistic Regression Recall: 0.7559174479450217
Logistic Regression Accuracy: 0.7617769827072153
